In [1]:
!pip install yahoo_fin
!pip install yahoo_fin --upgrade
!pip install dash
!pip install dash_table
!pip install colorlover
!pip install jupyter_dash

Requirement already up-to-date: yahoo_fin in /opt/anaconda3/lib/python3.7/site-packages (0.8.9.1)


In [4]:
import pandas as pd
import yahoo_fin as yfin 
import yahoo_fin.stock_info as st
import numpy as np
import matplotlib.pyplot as plt


# for Conditional Formatting
import dash
import dash_table
import dash_html_components as html
import colorlover
from jupyter_dash import JupyterDash

# Finance data  
def fin_data_points(tickets):    
    '''
    Computes finance data for the given list of assets (tickets)
    '''
    
    #tickets = ["MSFT","GM"]#,"BMW.DE","AAPL","NFLX","IBM"]

    def company_finance_fn(tbl_name,column_name):
        totalAssets_tbl = tbl_name.loc[column_name]
        totalAssets_maxdt = totalAssets_tbl.index.max()
        return totalAssets_tbl["endDate" == totalAssets_maxdt]    

    #create tbl:
    Company_Data = pd.DataFrame(columns = ['P/B','P/E','D/E','ROE','ROA',
                                       'NetProfit','GrossProfit','EBIT',
                                       'ProfitMargin', 'ForwardDividendYeild',
                                       'Name','Industry','Sector','Country', 'Currency'], index = tickets) 

    try:
        for i in range(len(tickets)):
            company = tickets[i]
        
            #get data from Yahoo Finance:
            info = st.get_company_info(company)
            forecast = st.get_analysts_info(company)
            balance_tbl = st.get_balance_sheet(company)
            income_tbl = st.get_income_statement(company)
            stats_tbl = st.get_stats(company)
            fins = st.get_financials(company, yearly = True, quarterly = False)
            fin_tbl = fins['yearly_income_statement']
            qd = st.get_quote_data(company)
        
            Company_Capitalization = qd['marketCap']
            Company_Assets = company_finance_fn(balance_tbl,"totalAssets")
            Company_Liabilities = company_finance_fn(balance_tbl,"totalLiab")
            Company_Equity =  Company_Assets - Company_Liabilities
            Company_NetProfit = company_finance_fn(income_tbl,"totalRevenue") #=totalRevenue 	
        
            #prepare fin data:
            PB = Company_Capitalization/abs(Company_Equity)
            PE = Company_Capitalization/Company_NetProfit
            #ForwardPE = qd['forwardPE']
            DE = Company_Liabilities/Company_Equity
            ROE = stats_tbl.iloc[34,1] #Return on Equity (ttm) 	
            ROA = stats_tbl.iloc[33,1]  #Return on Assets (ttm) 	
            NetProfit = company_finance_fn(income_tbl,"totalRevenue")
            GrossProfit = company_finance_fn(income_tbl,"grossProfit")
            Ebit = company_finance_fn(income_tbl,"ebit")
            ProfitMargin =  stats_tbl.iloc[31,1] #Profit Margin
            #DebtToEquity = stats_tbl.iloc[46,1] #Total Debt/Equity (mrq) 
            ForwardDividendYeild = stats_tbl.iloc[22,1] #Forward Annual Dividend Yield 4
        
            Name = qd['longName']
            Industry = info.loc['industry'].Value
            Sector = info.loc['sector'].Value   
            Country = info.loc['country'].Value  
            Currency = qd['currency']
        
            #Company_forecast_tbl = forecast['Growth Estimates']
        
            #fill tbl:
            Company_Data['P/B'].loc[company] = round(PB,4)
            Company_Data['P/E'].loc[company] = round(PE,4)
            #Company_Data['ForwardPE'].loc[company] = round(ForwardPE,4)
            Company_Data['D/E'].loc[company] = round(DE,2)
            Company_Data['ROE'].loc[company] = ROE
            Company_Data['ROA'].loc[company] = ROA
            Company_Data['NetProfit'].loc[company] = NetProfit
            Company_Data['GrossProfit'].loc[company] = GrossProfit
            Company_Data['EBIT'].loc[company] = Ebit
            Company_Data['ProfitMargin'].loc[company] = ProfitMargin
            #Company_Data['DebtToEquity'].loc[company] = DebtToEquity
            Company_Data['ForwardDividendYeild'].loc[company] = ForwardDividendYeild
        
            Company_Data['Name'].loc[company] = Name
            Company_Data['Industry'].loc[company] = Industry
            Company_Data['Sector'].loc[company] = Sector
            Company_Data['Country'].loc[company] = Country
            Company_Data['Currency'].loc[company] = Currency
    except:
        raise
    
    return Company_Data


def highlight_thebest_fin(data, color='green'):
    '''
    highlight the minimum in columns in "P/B","P/E","D/E"
    highlight the maximum in other columns
    '''
    
    min_v = ["P/B","P/E","D/E"]
    attr = 'background-color: {}'.format(color)
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        if data.name not in min_v:
            is_max = data == data.max()
            return [attr if v else '' for v in is_max]
        else:
            is_min = data == data.min()
            return [attr if v else '' for v in is_min]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index = data.index, columns = data.columns)



In [5]:
lt = ["MSFT","GM","BMW.DE","AAPL","NFLX","IBM"]
Company_Data = fin_data_points(lt)
Fin_Data = Company_Data.loc[:,'P/B':'ForwardDividendYeild']

In [6]:
#display finance data and highlight the best on each finance parameter
Fin_Data.style.apply(highlight_thebest_fin)

,P/B,P/E,D/E,ROE,ROA,NetProfit,GrossProfit,EBIT,ProfitMargin,ForwardDividendYeild
MSFT,15.810100,13.355200,1.350000,47.08%,13.76%,168088000000,115856000000,69916000000,36.45%,0.75%
GM,1.444800,0.586000,3.730000,24.88%,3.92%,122485000000,14502000000,8559000000,9.13%,0.00%
BMW.DE,0.836300,0.519800,2.520000,17.37%,3.43%,98990000000,12211000000,4837000000,9.93%,2.42%
AAPL,39.302200,9.354600,3.960000,127.13%,19.30%,274515000000,104956000000,66288000000,25.00%,0.54%
NFLX,23.846100,10.556200,2.550000,37.86%,9.72%,24996056000,9719737000,4585289000,15.92%,nan
IBM,5.999400,1.689000,6.530000,24.53%,3.84%,73621000000,35575000000,8584000000,7.17%,4.73%


In [7]:
#display general info
INFO_Data = Company_Data.loc[:,'Name':'Currency']
print(INFO_Data)

                                               Name  \
MSFT                          Microsoft Corporation   
GM                           General Motors Company   
BMW.DE  Bayerische Motoren Werke Aktiengesellschaft   
AAPL                                     Apple Inc.   
NFLX                                  Netflix, Inc.   
IBM     International Business Machines Corporation   

                               Industry                  Sector  \
MSFT            Software—Infrastructure              Technology   
GM                   Auto Manufacturers       Consumer Cyclical   
BMW.DE               Auto Manufacturers       Consumer Cyclical   
AAPL               Consumer Electronics              Technology   
NFLX                      Entertainment  Communication Services   
IBM     Information Technology Services              Technology   

              Country Currency  
MSFT    United States      USD  
GM      United States      USD  
BMW.DE        Germany      EUR  
AAPL    Unit